In [ ]:
import numpy as np
import torch
import torchvision.datasets
import torchvision.models
import torchvision.transforms
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
import torch.utils.data
import torch.nn as nn
from tqdm import tqdm, tnrange
import warnings
warnings.filterwarnings("ignore")
import copy

import pickle as pkl
import pandas as pd
from scipy.stats import entropy
CUDA = torch.cuda.is_available()
CUDA_DEVICE = 0
if CUDA:
    device='cuda:1'
else:
    device='cpu'
torch.cuda.is_available()

CUDA = torch.cuda.is_available()
CUDA_DEVICE = 0
try:
    from google.colab import drive
    drive.mount('/content/drive')
    base='drive/MyDrive'
except:
    if torch.cuda.device_count()>1:
        base='.'
    else:
        base='Google Drive'

if CUDA:
    device='cuda:1'
else:
    device='cpu'
torch.cuda.is_available()


Mounted at /content/drive


False

In [ ]:
class RBM(nn.Module):

    def __init__(self, n_vis, n_hid):
        """Create a RBM."""
        super(RBM, self).__init__()
        
        self.v_bias = nn.Parameter(torch.zeros(1, n_vis).to(device))
        self.h_bias = nn.Parameter(torch.zeros(1, n_hid).to(device))
        self.Weight = nn.Parameter(std*torch.randn(n_hid, n_vis).to(device))


    def v2h(self, v):
        return torch.sigmoid(F.linear(v, self.Weight, self.h_bias)).bernoulli()

    def h2v(self, h):
        return torch.sigmoid(F.linear(h, self.Weight.t(), self.v_bias)).bernoulli()
    
    def Fv(self, v):
        v_term = torch.matmul(v, self.v_bias.t()).view(len(v))
        h_term = torch.sum(F.softplus(F.linear(v, self.Weight, self.h_bias)), dim=1)
        return -h_term -v_term

    def energy(self, v, h):
        v=v.bernoulli()
        h=h.bernoulli()
        return -torch.matmul(v, self.v_bias.t())-torch.matmul(torch.matmul(v, self.Weight.t()),h.t())-torch.matmul(h, self.h_bias.t())


    def forward(self, v):

        return v.bernoulli()
from torch.utils.data import Dataset

class CustomDataset(Dataset): 
    def __init__(self, dataset):
        data_x = dataset
        self.x_data = data_x
#         self.y_data = data_y

    # 총 데이터의 개수를 리턴
    def __len__(self): 
        return len(self.x_data)
    # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
    def __getitem__(self, idx): 
        x = torch.FloatTensor(self.x_data[idx])
#         y = torch.FloatTensor([self.y_data[idx]])
        return x

def data_to_loader(fullconfigs):
    fulldata=CustomDataset(fullconfigs)
    full_dataset = fulldata
    full_loader = torch.utils.data.DataLoader(full_dataset, batch_size)
    return full_loader


def train_and_get_data(n_hid, model, lr, train_loader):
    # Train and get the new result
    rbm=RBM(n_vis, n_hid)
    train_op = optim.SGD(rbm.parameters(), lr, momentum=0.9)
    rbm.train()
    val_loss_list=[]
    model_list=[]
    # for epoch in tnrange(n_epochs):
    correction=0
    for epoch in tnrange(n_epochs):
        for _, (data) in enumerate(train_loader):
            v= data.view(-1, n_vis).to(device)
            Q_bf_list=torch.exp(-rbm.Fv(full_configs1)-correction).float()
            # while np.isnan(Q_bf_list[0].detach().numpy())==True:
            #     print('correction')
            #     correction=correction*10
            #     Q_bf_list=torch.exp(-rbm.Fv(v_list_ising2)-correction)

            Qv=torch.tensor(Q_bf_list/torch.sum(Q_bf_list)).to(device)
            Fv_Q=torch.dot(rbm.Fv(full_configs1), Qv)

            train_loss = torch.mean(rbm.Fv(v)) - Fv_Q
            train_op.zero_grad()
            train_loss.backward()
            train_op.step()
        if epoch in epoch_to_save:
            model_list.append(copy.deepcopy(rbm.cpu().state_dict()))
            rbm.to(device)
            FE=-torch.log(torch.sum(torch.exp(-rbm.Fv(full_configs1))))
            if FE==float('inf'):
                FE=-torch.log(torch.sum(torch.exp(-rbm.Fv(full_configs1))))
            Fv_P=torch.dot(rbm.Fv(full_configs1).double(), Pv.double())
            GE=Fv_P-FE-S
            print('epoch={epoch}, GE={GE}'.format(epoch=epoch, GE=GE))
            val_loss_list.append(GE)
    return model_list, val_loss_list

def CM_model(models):
    new_v_bias=0; new_h_bias=0; new_Weight=0
    for m in range(10):
        new_v_bias+=models[str(m)][-1]['v_bias']/10
        new_h_bias+=models[str(m)][-1]['h_bias']/10
        new_Weight+=models[str(m)][-1]['Weight']/10
    return {'v_bias':new_v_bias, 'h_bias':new_h_bias, 'Weight':new_Weight}  

def mean_Fv(Q_m, v0):
    value=0
    for m in range(10):
        rbm=RBM(n_vis, n_hid, k=1)
        rbm.load_state_dict(Q_m[str(m)])
        value+=rbm.Fv(v0)/10
    return value


def get_next_states(state0):
    occupied_spot=[]
    use_alpha=False
    use_beta=False

    for i in range(1,len(state0)+1):
        if state0[-i]=='1':
            occupied_spot.append(i)
    # print(occupied_spot)
    movable_spot=[]
    movable_count=0
    for j in range(len(occupied_spot)):
        if occupied_spot[j]+1 not in occupied_spot:
            movable_spot.append(occupied_spot[j])
            movable_count+=1
    # print(movable_count)

    next_states=[]
    for k in range(len(movable_spot)):
        state1=state0
        state1=list(state1)
        state1[-movable_spot[k]]='0'
        try:
            state1[-movable_spot[k]-1]='1'
            state1=''.join(state1[e] for e in range(len(state1)))
        except:
            # An outflow of a partible.
            use_beta=True
        state1=''.join(state1[e] for e in range(len(state1)))
        next_states.append(state1)
        # An inflow of a new particle.
    if state0[-1]=='0':
        use_alpha=True
        state1=state0
        state1=list(state1)
        state1[-1]='1'
        state1=''.join(state1[e] for e in range(len(state1)))
        next_states.append(state1)
    use_alpha_beta=[use_alpha, use_beta]

    factor_next_states={}
    for l in range(len(next_states)):
        try:
            factor_next_states[str(next_states[l])]=1/(len(next_states)-use_beta-use_alpha)
            # print('aa')
            # print(1/(len(next_states-use_alpha-use_beta)))
        except:
            None

        if int(list(str2arr(state0)-str2arr(next_states[l]))[-1])==-1:
            factor_next_states[str(next_states[l])]=alpha
        if int(list(str2arr(state0)-str2arr(next_states[l]))[0])==1:
            factor_next_states[str(next_states[l])]=beta
    return next_states, factor_next_states

def check_boundary(state0):
    use_alpha=False
    use_beta=False
    if state0[0]=='1':
        use_beta=True
    if state0[-1]=='0':
        use_alpha=True

    return use_alpha, use_beta

def dec2bin(integer, n_hid):
    string=bin(int(integer))[2:]
    list0=[int(d) for d in string]
    while len(list0)<n_hid:
        list0=[0]+list0
    a=np.array([list0])[0]
    b=''.join(str(e) for e in a)
    return b

def arr2str(array0):
    a=np.array([array0])[0]
    b=''.join(str(e) for e in a)
    return b


def str2arr(str0):
    b=[int(str0[i]) for i in range(len(str0))]
    b=np.array(b)
    return b

def get_ab(config0):
    a=config0.count(0)
    b=config0.count(1)
    return a,b


def opD(list0):
    tf='tf'
    if list0[-1]==0:
        del list0[-1]
        N=N-1
        factor=factor/alpha

    else:
        None

def opE(list0):
    tf='tf'
    if list0[0]==1:
        del list0[0]
        N=N-1
        factor=factor/beta
    else:
        None






def seperate(config_list):
    for j in range(len(config_list)):
        if '01' in arr2str(config_list[j]):
            config0=config_list[j]
            break
        else:
            None
    try:
        for i in range(len(config0)-1):
            if config0[i]==0 and config0[i+1]==1:
                config_list.append(del_comp(config0, i))
                config_list.append(del_comp(config0, i+1))
                config_list.remove(config0)
                break
            else:
                None
    except:
        None
def check(config_list):
    TF='01'
    for i in range(len(config_list)):
        if '01' in arr2str(config_list[i]):
            TF=True
            break
        else:
            TF=False
    return TF

def del_comp(list0, i):
    list1=list0.copy()
    del list1[i]
    return list1

def opS(list0):
    config_list=[list0]
    while check(config_list):
   #     print(config_list)
        seperate(config_list)

    return config_list

def get_ideal_dist():
    dist={}
    for i in range(2**Nv):
        flist=opS(list(str2arr(dec2bin(i,Nv))))    
        f=0
        for j in range(len(flist)):
            a,b=get_ab(flist[j])
            f+=((1/alpha)**a)*((1/beta)**b)

        dist[dec2bin(i,Nv)]=f
    s=sum(dist.values())
    for i in range(2**Nv):
        dist[dec2bin(i,Nv)]=dist[dec2bin(i,Nv)]/s
    return dist
    

In [ ]:
def TASEP_dynamics(state0, n_sample, interval):
    state_list=[]
    for step in range(n_sample*interval):
        r0=np.random.randint(0,N+1)
        if r0==0:
            if state0[-1]=='0':
                if np.random.rand()<alpha:
                    state0=str2arr(state0)
                    state0[-1]='1'
                else:None
            else:None
        elif r0==N:
            if state0[0]=='1':
                if np.random.rand()<beta:
                    state0=str2arr(state0)
                    state0[0]='0'
                else:None
            else:None
        else:
            if state0[-r0]=='1' and state0[-r0-1]=='0':
                state0=str2arr(state0)
                state0[-r0]='0'
                state0[-r0-1]='1'
        state0=arr2str(state0)
        # state0=str2arr(state0)
        if step%interval==0:
            state_list.append(state0)
    
    # staet counting
    dist={}
    for i in range(len(state_list)):
        dist[state_list[i]]=0
    for i in range(len(state_list)):
        dist[state_list[i]]+=1
    observed=list(dist.keys())
    for i in range(len(observed)):
        
        dist[observed[i]]=dist[observed[i]]/len(state_list)
        
    for i in range(len(state_list)):
        state_list[i]=torch.tensor(str2arr(state_list[i]))
    return torch.stack(state_list).float()
    # return dist

In [ ]:
vol=4096
Nv=9
N=Nv
seed=dec2bin(np.random.randint(0,Nv),Nv)

for alpha in [0.3, 0.5, 0.7]:
    for beta in [0.7]:
        lat=TASEP_dynamics(seed, vol, 10*Nv)
        with open('{base}/TASEP/test_alpha={alpha}_beta={beta}.pkl'.format(base=base, alpha=alpha, beta=beta), 'wb') as f:
            pkl.dump(lat, f)


In [ ]:
lr=0.1
n_vis=9
N=n_vis
Nv=n_vis
std=0.1
alpha=0.1
beta=0.1

epoch_to_save=[2**i for i in range(17)]
n_epochs=epoch_to_save[-1]+1
torch.set_printoptions(precision=10)

for alpha in [0.3, 0.5, 0.7]:
    for beta in [0.7]:
        prob_dict=get_ideal_dist()
        full_configs1=[]
        full_configs2=[]
        for i in range(2**N):
            full_configs1.append(str2arr(dec2bin(i,N)))
            full_configs2.append(str2arr(dec2bin(i,N))*2-1)
        full_configs1=torch.tensor(full_configs1).float()
        full_configs2=torch.tensor(full_configs2).float()
        for n_hid in [1]:  
            Nh=n_hid
            Pv=torch.tensor(list(prob_dict.values()))
            S=entropy(Pv)
            for vol in [4096]:
                batch_size=int(vol/2)
                dict_model={}
                dict_GE={}
                train_loader_list=[]; val_loader_list=[]
                for m in range(10):
                    seed=dec2bin(np.random.randint(0,N),N)
                    configs1=TASEP_dynamics(seed, vol, 10*Nv)
                    train_loader_list.append(data_to_loader(configs1))
                m=0
                while m<10:
                    model0, loss=train_and_get_data(n_hid, 0, lr=lr, train_loader=train_loader_list[m])
                    dict_model[str(m)]=model0
                    dict_GE[str(m)]=loss
                    if np.isnan(loss[-1].detach().numpy())==True:
                        None
                    else: 
                        dict_model[str(m)]=model0
                        dict_GE[str(m)]=loss
                        m=m+1
                    print(m)

            with open('{base}/TASEP/state_dict/model_Nh={Nh}_alpha={alpha}_beta={beta}_vol={vol}.pkl'.format(base=base, Nh=Nh, alpha=alpha, beta=beta, vol=vol), 'wb') as f:
                pkl.dump(dict_model, f)
            with open('{base}/TASEP/loss/GE_Nh={Nh}_alpha={alpha}_beta={beta}_vol={vol}.pkl'.format(base=base, Nh=Nh, alpha=alpha, beta=beta, vol=vol), 'wb') as f:
                pkl.dump(dict_GE, f)


  0%|          | 0/65537 [00:00<?, ?it/s]

epoch=1, GE=0.41851506523982795
epoch=2, GE=0.22351656106388695
epoch=4, GE=0.016742587861767966
epoch=8, GE=0.0626593293928508
epoch=16, GE=0.0029160885171535256
epoch=32, GE=0.002065972864997434
epoch=64, GE=0.001860344813197301
epoch=128, GE=0.0016868002399217374
epoch=256, GE=0.0015614143731328411
epoch=512, GE=0.0015421653348264286
epoch=1024, GE=0.001816407622744265
epoch=2048, GE=0.002090371961831927
epoch=4096, GE=0.00217790573504395
epoch=8192, GE=0.0022333459218844
epoch=16384, GE=0.002270337328928562
epoch=32768, GE=0.002297680584382178
epoch=65536, GE=0.0023171154644430203
1


  0%|          | 0/65537 [00:00<?, ?it/s]

epoch=1, GE=0.4235783432497495
epoch=2, GE=0.2330275421027883
epoch=4, GE=0.02165847268081933
epoch=8, GE=0.050779498556018154
epoch=16, GE=0.003824143808262903
epoch=32, GE=0.0015853541143284744
epoch=64, GE=0.002368733149614499
epoch=128, GE=0.0021686006386447687
epoch=256, GE=0.0020534281280468747
epoch=512, GE=0.0022502704851472544
epoch=1024, GE=0.0030031065039990423
epoch=2048, GE=0.003100326813341603
epoch=4096, GE=0.003170683881053904
epoch=8192, GE=0.0032084671281182864
epoch=16384, GE=0.0032237936658390254
epoch=32768, GE=0.0032365852248998905
epoch=65536, GE=0.0032594143763891026
2


  0%|          | 0/65537 [00:00<?, ?it/s]

...

epoch=1, GE=0.07441015002314444
epoch=2, GE=0.052612199048716946
epoch=4, GE=0.029919918544818458
epoch=8, GE=0.036680709483193574
epoch=16, GE=0.031226876381927227
epoch=32, GE=0.02908672314172378
epoch=64, GE=0.028943601710864186
epoch=128, GE=0.028584336638096453
epoch=256, GE=0.02788537751924025
epoch=512, GE=0.02807429059702926
epoch=1024, GE=0.026940837479505042
epoch=2048, GE=0.02553605843115303
epoch=4096, GE=0.024769199651751173
epoch=8192, GE=0.02439082485702926
epoch=16384, GE=0.024201593077394
epoch=32768, GE=0.02410408866383129
epoch=65536, GE=0.02405317967687992
8


  0%|          | 0/65537 [00:00<?, ?it/s]

epoch=1, GE=0.08039481216680411
epoch=2, GE=0.05352431144132552
epoch=4, GE=0.028582811519798135
epoch=8, GE=0.03964250176652229
epoch=16, GE=0.030477091081290375
epoch=32, GE=0.028372635961553705
epoch=64, GE=0.028322053987066553
epoch=128, GE=0.028195579463419662
epoch=256, GE=0.027333399614507314
epoch=512, GE=0.027019304591830107
epoch=1024, GE=0.025908049482742612
epoch=2048, GE=0.023898856104836774
epoch=4096, GE=0.023214013256822597
epoch=8192, GE=0.022910957362062945
epoch=16384, GE=0.022759101092543865
epoch=32768, GE=0.02267966928594145
epoch=65536, GE=0.02263782921180635
9


  0%|          | 0/65537 [00:00<?, ?it/s]

epoch=1, GE=0.08328119092841213
epoch=2, GE=0.05388651341671391
epoch=4, GE=0.028587179799118623
epoch=8, GE=0.04201398573232229
epoch=16, GE=0.030417615855393265
epoch=32, GE=0.028353115450422628
epoch=64, GE=0.02828901376131121
epoch=128, GE=0.02817512669573219
epoch=256, GE=0.02751380243617252
epoch=512, GE=0.027574174655200068
epoch=1024, GE=0.027881869616801502
epoch=2048, GE=0.025567079809603044
epoch=4096, GE=0.024412190240963305
epoch=8192, GE=0.024064713826969175
epoch=16384, GE=0.02391088454950019
epoch=32768, GE=0.023835431713158606
epoch=65536, GE=0.02379735245273551
10
